# 텍스트 전처리#3

#### 전처리 과정

## 6. 단어 색인화와 정수 인코딩(Integer Encoding)

#### 단어 집합(vocabulary)
- 서로 다른 단어들의 모임, 단어사전
- book, books와 같이 단어의 변형 형태도 다른 단어로 간주함
- 예. 텍스트에 5000개 단어가 존재한다면 단어 집합의 크기는 5000이 됨

#### 단어에 정수인덱스 부여

- 단어집합의 단어들을 고유한 정수 인덱스를 랜덤하게 부여하거나, 단어 등장 빈도 기준으로 정렬한 뒤 부여함

### 단어 등장 빈도에 따른 단어 색인화와 문장의 정수 인코딩

#### 1) dictionary 사용

In [ ]:
!pip uninstall jedi

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [24]:
raw_text = "A barber is a person. a barber is good person. \
a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. \
Huge secret. His barber kept his word. a barber kept his word. \
His barber kept his secret. But keeping and keeping such a huge secret \
to himself was driving the barber crazy. the barber went up a huge mountain."

- 문장 토큰화

In [25]:
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


- 단어 토큰화 후 정제&정규화

In [30]:
sentences = sent_tokenize(raw_text)
# 불용어 목록
stop_words = set(stopwords.words('english'))
voca = {}
preprocess_sentences = []

# 단어 토큰화와 정규화, 불용어 처리
for sentence in sentences:
    words = word_tokenize(sentence)
    result = []
    for word in words:
        # 정규화: 소문자로변환
        word = word.lower()   
        # 불용어 처리
        if word not in stop_words:
            # 단어 길이가 2 이하인 경우 제거
            if len(word) > 2:   
                result.append(word)
                # 단어집합 생성, 빈도계산
                if word not in voca:
                    voca[word] = 0
                voca[word] += 1
    preprocess_sentences.append(result)
print(preprocess_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


- 단어집합 출력

In [31]:
print(f'단어집합: ', voca)

단어집합:  {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


- 단어집합을 빈도수가 높은 순서대로 정렬

In [32]:
# key -> 정렬의 기준
vocab_sorted = sorted(voca.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


- 높은 빈도수를 갖는 단어 순으로 **정수인덱스 부여**

In [36]:
word_to_index = {}
i = 1
for word, freq in vocab_sorted:
    if freq > 1:
        word_to_index[word] = i
        i += 1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


- 상위 5개 단어만 사용

In [37]:
vocab_size = 5

words_freq = [word for word, index in word_to_index.items() if index >= vocab_size + 1]
print(f'삭제할 단어들: ',words_freq)

# 인덱스가 5초과인 단어들의 인덱스 정보 삭제
for wo in words_freq:
    del word_to_index[wo]

print(word_to_index)

삭제할 단어들:  ['word', 'keeping']
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- 단어집합에 없는 단어(Out-Of_Vocabulary, OOV) 문제 해결을 위한 인덱스

In [38]:
# 전처리된 문장들
preprocess_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [41]:
word_to_index['OOV'] = len(word_to_index) + 1
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}

- 단어에 부여된 정수 인덱스를 사용하여 전처리된 문장들의 모든 단어들을 **정수 인코딩**

In [42]:
encoded_sentences = []

for sentence in preprocess_sentences:
    encoded_sent = []
    for word in sentence:
        try:
            # 단어집합에 있는 단어의 경우
            encoded_sent.append(word_to_index[word])
        except KeyError:
            # 단어집합에 없는 단어의 경우
            encoded_sent.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sent)
encoded_sentences

[[1, 5],
 [1, 6, 5],
 [1, 3, 5],
 [6, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [6, 6, 3, 2, 6, 1, 6],
 [1, 6, 3, 6]]

#### 2) Counter 사용

- 파이썬 내장 모듈인 collections의 Counter클래스 사용
- https://docs.python.org/3/library/collections.html

In [12]:
preprocess_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

- 문장별 단어들을 하나의 리스트로 병합

In [47]:
# , [] -> 을 안쓰면 int가 아니어서 sum() 불가능 [] -> 리스트라고 알려주는 기능
print(sum(preprocess_sentences, []))

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [64]:
import numpy as np
all_words_list = list(sum(preprocess_sentences, []))
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


- Counter()를 이용한 단어별 빈도수 계산

In [57]:
from collections import Counter

vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [58]:
vocab['barber']

8

- 빈도수가 높은 상위 5개 단어만 단어집합으로 저장

In [59]:
# most_common -> 상위 빈도 가져오기
vocab_freq = 5
vocab = vocab.most_common(vocab_freq)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

- 빈도수가 높은 단어순으로 정수 인덱스 부여

In [60]:
word_to_index = {}
i = 1
for word, freq in vocab:
    word_to_index[word] = i
    i +=1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- enumerate 사용하여 정수 인덱스 부여

In [19]:
list(enumerate(vocab))

[(0, ('barber', 8)),
 (1, ('secret', 6)),
 (2, ('huge', 5)),
 (3, ('kept', 4)),
 (4, ('person', 3))]

In [61]:
word_to_index = {}
for i, word_freq in enumerate(vocab):
    word_to_index[word_freq[0]]= i + 1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


#### 3) NLTK의 FreqDist 사용

In [62]:
from nltk import FreqDist

vocab = FreqDist(sum(preprocess_sentences,[]))
vocab

FreqDist({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, ...})

In [22]:
vocab['barber']

8

In [65]:
vocab_freq = 5
vocab = vocab.most_common(vocab_freq)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [24]:
word_to_index = {word_freq[0] : index+1 for index, word_freq in enumerate(vocab)}
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}

### Keras의 텍스트전처리
- Keras는 기본적인 전처리 도구들 제공
- Tokenizer() : 정수인코딩

In [66]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [67]:
tf.__version__

'2.16.1'

In [27]:
preprocess_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

- Tokenizer()를 이용하여 정수인코딩

In [70]:
# 단어별 인덱스, 빈도 작업
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocess_sentences)

- 단어별 인덱스 : word_index

In [71]:
# 단어별 인덱스 확인
tokenizer.word_index

{'barber': 1,
 'secret': 2,
 'huge': 3,
 'kept': 4,
 'person': 5,
 'word': 6,
 'keeping': 7,
 'good': 8,
 'knew': 9,
 'driving': 10,
 'crazy': 11,
 'went': 12,
 'mountain': 13}

- 단어별 빈도 확인 : word_counts

In [72]:
# 단어별 빈도 확인
tokenizer.word_counts

OrderedDict([('barber', 8),
             ('person', 3),
             ('good', 1),
             ('huge', 5),
             ('knew', 1),
             ('secret', 6),
             ('kept', 4),
             ('word', 2),
             ('keeping', 2),
             ('driving', 1),
             ('crazy', 1),
             ('went', 1),
             ('mountain', 1)])

- 각 단어를 정해진 인덱스로 변환 : texts_to_sequences()

In [73]:
encoded = tokenizer.texts_to_sequences(preprocess_sentences)
encoded

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

- Tokenizer(num_words=)를 사용하여 상위 5개 단어만 정수 인코딩

In [85]:
vocab_freq = 5
tokenizer1 = Tokenizer(num_words=vocab_freq + 1)
tokenizer1.fit_on_texts(preprocess_sentences)

In [81]:
tokenizer1.word_index

{'barber': 1,
 'secret': 2,
 'huge': 3,
 'kept': 4,
 'person': 5,
 'word': 6,
 'keeping': 7,
 'good': 8,
 'knew': 9,
 'driving': 10,
 'crazy': 11,
 'went': 12,
 'mountain': 13}

In [86]:
tokenizer1.word_counts

OrderedDict([('barber', 8),
             ('person', 3),
             ('good', 1),
             ('huge', 5),
             ('knew', 1),
             ('secret', 6),
             ('kept', 4),
             ('word', 2),
             ('keeping', 2),
             ('driving', 1),
             ('crazy', 1),
             ('went', 1),
             ('mountain', 1)])

In [35]:
encoded = tokenizer.texts_to_sequences(preprocess_sentences)
encoded

[[1, 5],
 [1, 5],
 [1, 3, 5],
 [2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4],
 [1, 4],
 [1, 4, 2],
 [3, 2, 1],
 [1, 3]]

- Tokenizer(oov_token='OOV')를 사용하여 정수인코딩

In [36]:
vocab_size = 5
tokenizer = Tokenizer(num_words=vocab_size + 2, oov_token='OOV')
tokenizer.fit_on_texts(preprocess_sentences)
print(tokenizer.word_index)
encoded = tokenizer.texts_to_sequences(preprocess_sentences)
encoded

{'OOV': 1, 'barber': 2, 'secret': 3, 'huge': 4, 'kept': 5, 'person': 6, 'word': 7, 'keeping': 8, 'good': 9, 'knew': 10, 'driving': 11, 'crazy': 12, 'went': 13, 'mountain': 14}


[[2, 6],
 [2, 1, 6],
 [2, 4, 6],
 [1, 3],
 [3, 5, 4, 3],
 [4, 3],
 [2, 5, 1],
 [2, 5, 1],
 [2, 5, 3],
 [1, 1, 4, 3, 1, 2, 1],
 [2, 1, 4, 1]]

----

## 7. 패딩(padding)

- 서로 다른 문장의 길이를 동일하게 맞춰주는 작업

In [87]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

preprocess_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [88]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocess_sentences)
encoded = tokenizer.texts_to_sequences(preprocess_sentences)
encoded

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

#### 1) Numpy로 패딩하기

In [91]:
max_len = max(len(item) for item in encoded)
print(f'최대길이: ', max_len)

최대길이:  7


In [40]:
for sentence in encoded:
    while len(sentence) < max_len:
        sentence.append(0)

padded_np = np.array(encoded)
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

#### 2) Keras 전처리도구 pad_sequences()를 이용하여 패딩하기

In [92]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded = tokenizer.texts_to_sequences(preprocess_sentences)
encoded

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

In [93]:
padded = pad_sequences(encoded)
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]])

In [43]:
# padding -> 패딩의 순서를 바꿀 때
padded = pad_sequences(encoded, padding='post')
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

In [44]:
(padded == padded_np).all()

True

- pad_sequnces(maxlen=)을 이용한 패딩

In [45]:
padded = pad_sequences(encoded, padding='post', maxlen=5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]])

- pad_sequnces(value=)를 이용한 패딩 : 0이 아닌 다른 숫자로 패딩

In [46]:
last_value = len(tokenizer.word_index) + 1
print(last_value)
padded = pad_sequences(encoded, padding='post', value=last_value)
padded

14


array([[ 1,  5, 14, 14, 14, 14, 14],
       [ 1,  8,  5, 14, 14, 14, 14],
       [ 1,  3,  5, 14, 14, 14, 14],
       [ 9,  2, 14, 14, 14, 14, 14],
       [ 2,  4,  3,  2, 14, 14, 14],
       [ 3,  2, 14, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  2, 14, 14, 14, 14],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13, 14, 14, 14]])

## 8. 원-핫 인코딩(One-Hot Encoding)

- 단어 집합의 크기를 벡터 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1을 부여하고 다른 인덱스에는 0을 부여하여 벡터로 표현하는 방식
- 원-핫 벡처(One-Hot Vector)

In [47]:
from konlpy.tag import Okt

text = '나는 텍스트 전처리를 배우고 있다'
okt = Okt()
tokens = okt.morphs(text)
print(tokens)

['나', '는', '텍스트', '전', '처리', '를', '배우고', '있다']


In [ ]:
word_to_index = {word : index for index, word in enumerate(tokens)}
print(f'단어집합 :', word_to_index)

In [49]:
def one_hot_encoding(word, word_to_index):
    one_hot_vector = [0]*len(word_to_index)
    index = word_to_index[word]
    one_hot_vector[index] = 1
    return one_hot_vector

In [50]:
one_hot_encoding('텍스트', word_to_index)

[0, 0, 1, 0, 0, 0, 0, 0]

In [ ]:
for word in tokens:
    print(one_hot_encoding(word, word_to_index))

#### Keras를 이용한 원-핫 인코딩

In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('단어집합:', tokenizer.word_index)

단어집합: {'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


In [52]:
encoded = tokenizer.texts_to_sequences([text])
encoded

[[4, 2, 5, 1, 2, 6, 3, 1, 1, 3, 7]]

In [53]:
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded2 = tokenizer.texts_to_sequences([sub_text])
encoded2

[[2, 5, 1, 6, 3, 7]]

In [54]:
to_categorical(encoded[0])

array([[0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
to_categorical(encoded2[0])

-----